# Loading data for the S&P500 and analyzing its constituents

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import Counter
from typing import List, Optional
from BSquant import load_and_process_data
from BSquant import cs_pattern_recognition
from BSquant import cs_performance
from BSquant import plot_cs_performance

pd.set_option("display.max_columns", None)
%load_ext autoreload
%autoreload 2

In [ ]:
# Define the path to your file
file_path_to_ticker_data = "./../data/SP500_tickers_one_per_line.txt"

tickers = []

# Open the ticker-file with a context manager and read each line adding ot to the list of tickers
with open(file_path_to_ticker_data, "r") as file:
    for line in file:
        ticker = line.strip()  # Strip newline characters and whitespace
        tickers.append(ticker)  # Add the cleaned ticker to the list

print("Number of tickers (may include multiple tickers per stock) is", len(tickers))
print("Number of unique tickers is:", set(tickers).__len__())

In [ ]:
for i, ticker in enumerate(tickers):
    print(f"{i+1}:{ticker}")

Note: the first `.dropna()` is agressive , though it will be applied row wise only and hence treat each stock "fairly". MSFT will have a longer history than AMZN. I think that is a good compromise. 

`df[['DlyPrc', 'DlyClose']].dropna().eval('DlyPrc == DlyClose').all() # DlyPrc is equal to DlyClose`
verifies that `CRSP's` `DlyPrc` is indeed equal to `DlyClose` whenever we got an entry. Should be be interested to work only with closing prices for your project, obraining the entries for `DlyPrc` would have been sufficient.

Positive surprise: I think we can retrieve VWAP like so (confirm with CRSP): 

`df['VWAP'] = df['DlyPrcVol'] / df['DlyVol']`

## Taking a look at missing data

In [ ]:
rename_map = {
    "Ticker": "ticker",
    "DlyCalDt": "date",
    "DlyPrc": "prc",
    "DlyOpen": "open",
    "DlyHigh": "high",
    "DlyLow": "low",
    "DlyClose": "close",
    "DlyVol": "vol",
    "DlyPrcVol": "price_vol",
}

usecols = [
    "Ticker",
    "DlyCalDt",
    "DlyPrc",
    "DlyOpen",
    "DlyHigh",
    "DlyLow",
    "DlyClose",
    "DlyVol",
    "DlyPrcVol",
]

df_missing_data = (
    pd.read_csv(
        "./../data/SP500_daily_data_stock2.csv.gz", usecols=usecols, compression="gzip"
    )
    .rename(columns={k: v for k, v in rename_map.items() if k in usecols})
    .fillna(999999)
)

In [ ]:
df_missing_data = (
    pd.read_csv(
        "./../data/SP500_daily_data_stock2.csv.gz",
        usecols=[
            "Ticker",
            "DlyCalDt",
            "DlyPrc",
            "DlyOpen",
            "DlyHigh",
            "DlyLow",
            "DlyClose",
            "DlyVol",
            "DlyPrcVol",
        ],
        compression="gzip",
    )
    .rename(columns={k: v for k, v in rename_map.items() if k in usecols})
    .fillna(999999)
)

In [ ]:
%%time 

# dg[dg.apply(lambda row: (row == 999999).any(), axis=1)]  # takes ages
df_missing_data[(df_missing_data == 999999).any(axis=1)]  # done in about 500 ms

In [ ]:
# why do we have high, low and close data, but not open data?
df_missing_data[(df_missing_data == 999999).any(axis=1)].query(
    " ticker == 'AAPL' "
).tail(20)

In [ ]:
df_missing_data.shape

## Remove rows with missing data

In [ ]:
%%time
df = load_and_process_data(
    file_path="./../data/SP500_daily_data_stock2.csv.gz",
    usecols=[
        "Ticker",
        "DlyCalDt",
        "DlyPrc",
        "DlyOpen",
        "DlyHigh",
        "DlyLow",
        "DlyClose",
        "DlyVol",
        "DlyPrcVol",
    ],
    compression="gzip",
    #                           selected_start_date=pd.Timestamp(2021, 1, 1),
    #                           selected_end_date=pd.Timestamp(2022, 12, 31)
)

df

In [ ]:
%%time

file_path = "./../data/SP500_daily_data_stock2.csv.gz"
usecols = [
    "Ticker",
    "DlyCalDt",
    "DlyPrc",
    "DlyOpen",
    "DlyHigh",
    "DlyLow",
    "DlyClose",
    "DlyVol",
    "DlyPrcVol",
]
compression = "gzip"
df = load_and_process_data(
    file_path=file_path, usecols=usecols, compression=compression
)

In [ ]:
df.shape

In [ ]:
len(df) / len(df_missing_data)  # we drop 20% of the data.

Removing 20% of the data is quite significant. Yet, it is a steight-forward approach of cleaning up missing data. A more careful approach should investigate why the data are missing in the first palce and whether they can be inferred by another source or interpolated.

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df[
    "ticker"
].unique().__len__()  # we have data for exactly 500 stocks; for stocks we do not have data, but that doesnt matter for our purpose

## Let us now compute for how many days per stock we got data for. 

Companies and be included and taken off the S&P 500 index. Some startups that were not previously listed might prosper and develop into companies large enough to be included in the index, while other may be outcompeted by others, cease to exist, acquired, split up, or taken private and hence either be excluded and/or delisted. Let us investigate for how many days we got data for each stock. While doing so, we find an interesting detour related to Python performance:

Python is primarily considered an interpreted language. Python code is executed by an interpreter, which reads the code at runtime and executes it line by line. This process is different from compiled languages, where the source code is transformed into machine code or bytecode before execution, typically resulting in an executable file. However, at a more detailed level, Python code is indeed compiled under the hood. More precisely, when Python code is executed, it is compiled into bytecode, which is a lower-level, platform-independent representation of the source code. This bytecode is then interpreted by the `Python Virtual Machine (PVM)`, however, compared to a purely compiled language such as `C` or `C++`, not turned into a standalone executable file. This process is automatic and transparent to the user, making Python feel like a purely interpreted language. Tools and third-party packages do exist that can package Python programs along with an interpreter into a standalone executable, but this is an additional step beyond Python's standard behavior.

The important point is that parsing byte-code through the PVM imposes an overhead which costs time. Hence, `Python` is considered "slow". However, you can use `C` and `C++` code within `Python` to leverage performance benefits. This is a common practice for computational heavy tasks where the execution speed of `Python` is a bottleneck. Integrating `C` or `C++` code into `Python` can significantly improve the performance of certain operations, especially those that are CPU-bound, such as numerical computations, data processing, and more.
This, however, required more detailed knowlege of the `Python compiler`, is not straight forward, and a topic for another repository. 

However, that does not mean we cannot speed up our code. Paricularly, we can make use of libraries that are written, at least partially, in `C` and available in Python, such as `numpy`. As `pandas` makes use of `numpy`, it is often possible to enjoy better performance, especially when we compute in-momory like we using `pandas`. Thus, it is generally good advise for the sake of performance, to "write highl-level code thinking low-level", and the following is meant to demonstrate this.

To compute the number of days we got for each of the S&P500 members, a streight-forward (but slow) method is to loop trough each ticker, filter the data frame according to the ticker, and to compute the number of rows. This will be executed below.

In [ ]:
%%time 

days_per_ticker = {}

for ticker in tickers:
    days_per_ticker[ticker] = df.query("ticker == @ticker").shape[
        0
    ]  # takes about 16.8 seconds
#     days_per_ticker[ticker] = df[df['ticker'] == ticker].shape[0]  # takes about 59.6 seconds and is three times slower, still.

As you see this step took approximately 16.8 seconds on the machine this code was executed on. Making use of the `pandas` native `.goupby()` method, which is written in `C`, and storing the results in a dictionary, achieves the same task in just about 112 ms, i.e, the computation is 150 times, i.e. an order of magnitude 
faster.  

In [ ]:
%%time 
days_per_ticker = df.groupby("ticker").size().to_dict();

## We now investigate how the length of the history of each stock [in days] is distributed

In [ ]:
plt.figure(figsize=(7, 7))
plt.hist(list(days_per_ticker.values()), bins=30)
plt.show()

In [ ]:
# Counter objects are a part of the collections module in Python's standard library.
# They are specialized dictionary subclasses designed to count hashable objects.
# A Counter is a collection where elements are stored as dictionary keys and their counts are stored
# as dictionary values.

Counter(list(days_per_ticker.values())).most_common(3)[
    0
]  # Counter(list(days_per_ticker.values())).most_common(3)[0][0] then extracts the number that occurs most often.

# 81 stocks contain 7881 days of data

## How are the stocks weighted with repect to the one with the longst history in the portfolio?

In [ ]:
max_days_ticker = max(
    days_per_ticker, key=days_per_ticker.get
)  # find the ticker with the maximum number of days
max_days = days_per_ticker[
    max_days_ticker
]  # retrieve the value (number of days) for this ticker
print(
    f"The ticker with the maximum number of days is: {max_days_ticker}, with {max_days} days."
)

max_days = max(days_per_ticker.values())  # find the maximum number of days
weights_per_ticker = {
    ticker: days / max_days for ticker, days in days_per_ticker.items()
}  # Calculate the weight for each ticker
weights_per_ticker

# z-transform the weights.

## Alternatively, we could define a start and end date ourselves and make sure to select only those stocks with a densely populated history.

Densely here means that the stocks should have the same number of data. This ensures stocks thate were recently taken in are not selected as they do not contain enough data

In [ ]:
selected_start_date = pd.Timestamp(2012, 1, 1)
selected_end_date = pd.Timestamp(2022, 12, 31)
df_filtered = df[
    (df["date"] >= selected_start_date) & (df["date"] <= selected_end_date)
]
df_filtered.groupby(
    "ticker"
).size().value_counts()  # 374 out of the 500 stocks are of the desired duration

If we wanted to just go for the mode directly, we could have achieved this by

In [ ]:
mode_size = df_filtered.groupby("ticker").size().mode()[0]
mode_size

And counted the number of stocks of that lenth using

In [ ]:
df_common_size = (
    df_filtered.groupby("ticker")
    .filter(lambda x: len(x) == mode_size)
    .reset_index(drop=True)
)

df_common_size["ticker"].unique().__len__()

If we selected only those stocks that have an equal amount of days between our start and end day, we have to reduce our universe from 500 stocks to 374. This is a significant reduction that one should be sure to afford.

As an alternative way, we appreaciate the different length of the data and conduct the pattern analysis for each of them separately.

# How does each stock evolve in time?

We limit ourselves to onem year of data to see how each of the stocks in the portfolio performed relative to their starting price

In [ ]:
def add_normalized_price(df: pd.DataFrame) -> pd.DataFrame:
    df["first_price_indicator"] = np.where(df.index == 0, 1, 0)
    df["first_price_value"] = df["first_price_indicator"] * df["close"]
    df["first_price_value"].replace(to_replace=0, method="ffill", inplace=True)
    df["normalized_price"] = df["close"] / df["first_price_value"]
    df.drop(columns=["first_price_indicator", "first_price_value"], inplace=True)
    return df

In [ ]:
# implementation using a multi-indexed data frame
result = df_common_size.set_index(["ticker", "date"]).join(
    df_common_size.groupby("ticker").first().add_prefix("first_")
)  # dg.set_index(['Date','ListingId']) will be equivalent to the vectorized version
result["normalized_price"] = result["close"] / result["first_close"]

In [ ]:
# plotting the data. Note you can select a nunber of stocks via the variable "counter" as well.

selected_start_date = pd.Timestamp(2022, 1, 1)
selected_end_date = pd.Timestamp(2022, 12, 31)

df_filtered = df[
    (df["date"] >= selected_start_date) & (df["date"] <= selected_end_date)
]
df_filtered

mode_size = df_filtered.groupby("ticker").size().mode()[0]

df_common_size = (
    df_filtered.groupby("ticker")
    .filter(lambda x: len(x) == mode_size)
    .reset_index(drop=True)
)

result = df_common_size.set_index(["ticker", "date"]).join(
    df_common_size.groupby("ticker").first().add_prefix("first_")
)  # dg.set_index(['Date','ListingId']) will be equivalent to the vectorized version
result["normalized_price"] = result["close"] / result["first_close"]

plt.figure(figsize=(10, 6))

counter = 0

for ticker, data in result.groupby(level="ticker"):
    plt.plot(
        data.index.get_level_values("date"), data["normalized_price"], label=ticker
    )
#     counter += 1
#     if counter == 20:
#         break

# plt.legend(title='Ticker', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.title("Normalized price by date for the selected stocks")
plt.xlabel("date")
plt.ylabel("normalized price")
plt.tight_layout()
plt.show()

We see that the stock prices are not adapted to stock splits. As we work with intraday returns, we get away dealing with price-adjustments which, strictly speaking, is a topic in its own right. However, we do see the investment universe evolved, seemingly randomly, and there were constituents that performed positively, neutral, and negatively. 

Hence, selecting one stock in hindsight, and evaluating its buy-and-hold performance, is subject to bias. We will test in the following, how the candlestick patterns perform for the investment universe, to see whether they allow a a portfolion to be actively managed in a long-short fashion. 

# Candlestick analysis

## The pipeline pattern

In [ ]:
# i) is there a smarter way of passing input arguments to load_and_process_data?

# ii) can I somehow cache the data frame rather than reading it from disk every single time?


def data_processing_pipeline(
    file_path: str = "./../data/SP500_daily_data_stock2.csv.gz",
    usecols: List[str] = [
        "Ticker",
        "DlyCalDt",
        "DlyPrc",
        "DlyOpen",
        "DlyHigh",
        "DlyLow",
        "DlyClose",
        "DlyVol",
        "DlyPrcVol",
    ],
    compression: Optional[str] = "gzip",
    ticker: Optional[str] = None,
    selected_start_date: Optional[pd.Timestamp] = None,
    selected_end_date: Optional[pd.Timestamp] = None,
) -> pd.DataFrame:
    df = load_and_process_data(
        file_path, usecols, compression, ticker, selected_start_date, selected_end_date
    )

    steps = [cs_pattern_recognition, cs_performance]

    for step in steps:
        df = step(df)

    return df

In [ ]:
%%time
data_processing_pipeline(
    ticker="AAPL",
    selected_start_date=pd.Timestamp(2020, 1, 1),
    selected_end_date=pd.Timestamp(2022, 12, 31),
)

In [ ]:
ticker = tickers[0]
df.query("ticker == @ticker")
cs_pattern_recognition(df=df.query("ticker == @ticker"))

In [ ]:
!free -h

In [ ]:
selected_start_date = pd.Timestamp(2021, 1, 1)
selected_end_date = pd.Timestamp(2022, 12, 31)

df_filtered = df[
    (df["date"] >= selected_start_date) & (df["date"] <= selected_end_date)
]
df_filtered

In [ ]:
%%time
cs_pattern_recognition(df=df_filtered)

In [ ]:
cs_performance(cs_signals_df=cs_pattern_recognition(df=df_filtered)).query(
    "ci_lower > 0.5"
)

In [ ]:
ax = cs_performance(cs_signals_df=cs_pattern_recognition(df=df_filtered))[
    ["ci_upper", "TP_wilson", "ci_lower"]
].plot(figsize=(10, 10), rot=90)
plt.axhline(y=0.5, color="r", linestyle="--")
plt.show()

In [ ]:
!free -h

In [ ]:
plot_cs_performance(
    cs_performance(cs_pattern_recognition(df=df.query("ticker == @ticker")))
)

In [ ]:
plot_cs_performance

# Can we run this on HPC

# Appendix

In [ ]:
# pytables; pychunks; loading into batches? Is it possible to batch data in pandas?

In [ ]:
x = np.array([[0.3, 0.2, 0.15]]).T  # x is a column vector
x

In [ ]:
x.shape

In [ ]:
x.T

In [ ]:
x.T.shape

In [ ]:
x @ x.T

In [ ]:
np.linalg.matrix_rank(x @ x.T)